# 6. Figures

## Import Packages

In [ ]:
import matplotlib.pyplot as plt


def layout_subfigures(size, subfigures_dict):
    """
    Creates a matplotlib figure with subfigures arranged based on positions in inches.

    Parameters:
    - size: tuple of (width, height) for the overall figure size in inches.
    - subfigures_dict: dictionary where keys are subfigure names and values are dictionaries
                       containing 'position' as a tuple (x, y, width, height) in inches and
                       'plot_func' as the function to create the specific subfigure.

    Returns:
    - fig: the matplotlib figure object.
    - axes_dict: a dictionary where keys are the subfigure names and values are the corresponding axes.

    Example structure for subfigures_dict:
    {
        'subfig1': {'position': (1, 2, 3, 4), 'plot_func': plot_func1},
        'subfig2': {'position': (5, 6, 2, 3), 'plot_func': plot_func2},
        ...
    }
    """
    # Create the main figure with the specified size
    fig = plt.figure(figsize=size)
    axes_dict = {}

    for name, subfig_data in subfigures_dict.items():
        position = subfig_data["position"]  # (x, y, width, height) in inches

        # Convert inches to figure-relative coordinates
        left = position[0] / size[0]
        bottom = position[1] / size[1]
        width = position[2] / size[0]
        height = position[3] / size[1]

        # Add an axes to the figure at the specified location
        ax = fig.add_axes([left, bottom, width, height])

        # Store the axes in the dictionary with the corresponding name
        axes_dict[name] = ax

    return fig, axes_dict


In [ ]:
# Dictionary defining the subfigures with positions in inches
subfigures_dict = {
    "AFM_Image": {"position": (1, 1, 4, 3)},
    "subfig2": {"position": (6, 1, 3, 2)},
}

# Create a custom figure of size (10 inches width, 6 inches height)
layout_subfigures(size=(, 6), subfigures_dict=subfigures_dict)


In [ ]:
import sys

sys.path.append("../../")
sys.path.append('../../src')

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import random
import matplotlib.pyplot as plt

from m3_learning.nn.random import random_seed
from m3_learning.viz.style import set_style
from m3_learning.viz.printing import printer
from m3_learning.be.viz import Viz
from m3_learning.be.dataset import BE_Dataset
from m3_learning.be.nn import SHO_fit_func_nn, SHO_Model
from m3_learning.nn.Fitter1D.Fitter1D import Multiscale1DFitter, Model
from m3_learning.nn.Fitter1D.Fitter1D import ComplexPostProcessor
from m3_learning.viz.layout import (
    inset_connector,
    add_box,
    labelfigs
)
import m3_learning

from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg

printing = printer(basepath = './Figures/')


set_style("printing")
random_seed(seed=42)

%matplotlib inline

## Loads Data

In [ ]:
# Specify the filename and the path to save the file
filename = "./data_raw_unmod.h5"
save_path = "./Data"

data_path = save_path + "/" + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path, SHO_fit_func_LSQF=SHO_fit_func_nn)

# print the contents of the file
dataset.print_be_tree()

## Figure 1

In [ ]:
# instantiates the visualization object
BE_viz = Viz(dataset, printing, verbose=True)

In [ ]:
fig = BE_viz.raw_be(dataset, filename="Figure_2_raw_be_experiment")

In [ ]:
axes = fig.axes
axes

In [ ]:
def copy_axes_properties(source_ax, target_ax, secondary_ax=None, type_ax=None):
    """Copy properties and data from source_ax to target_ax."""
    target_ax.set_xlabel(source_ax.get_xlabel())
    target_ax.set_ylabel(source_ax.get_ylabel())
    target_ax.set_xlim(source_ax.get_xlim())
    target_ax.set_ylim(source_ax.get_ylim())
        
    for line in source_ax.get_lines():
        target_ax.plot(line.get_xdata(), line.get_ydata(), color=line.get_color())
        
    if secondary_ax != None:
        if type_ax == 'twin':
            ax_twin = target_ax.twinx()
            for line in secondary_ax.get_lines():
                ax_twin.plot(line.get_xdata(), line.get_ydata(), color=line.get_color())
        elif type_ax == 'same':
            for line in secondary_ax.get_lines():
                target_ax.plot(line.get_xdata(), line.get_ydata(), color=line.get_color())
                
                max_x_lim = max(source_ax.get_xlim()[1], secondary_ax.get_xlim()[1])
                min_x_lim = min(source_ax.get_xlim()[0], secondary_ax.get_xlim()[0])
                max_y_lim = max(source_ax.get_ylim()[1], secondary_ax.get_ylim()[1])
                min_y_lim = min(source_ax.get_ylim()[0], secondary_ax.get_ylim()[0])
                target_ax.set_xlim((min_x_lim, max_x_lim))
                target_ax.set_ylim((min_y_lim, max_y_lim))
        else:
            ax_new = target_ax.inset_axes([0.5, 0.65, 0.48, 0.33])
            x_start = 120
            x_end = 140
            ax_new.plot(dataset.hysteresis_waveform)
            ax_new.set_xlim(x_start, x_end)
            ax_new.set_ylim(0, -15)

            # drows the inset connector
            inset_connector(
                source_ax,
                target_ax,
                ax_new,
                [(x_start, 0), (x_end, 0)],
                [(x_start, 0), (x_end, 0)],
                color="k",
                linestyle="--",
                linewidth=0.5,
            )

            # adds a box on the figure
            add_box(
                target_ax,
                (x_start, 0, x_end, -15),
                edgecolor="k",
                linestyle="--",
                facecolor="none",
                linewidth=0.5,
                zorder=10,
            )
            ax_new.set_xlabel("Voltage Steps")
            ax_new.set_ylabel("Voltage (V)")

# Create a figure
fig = plt.figure(figsize=(14, 12))

# Define the GridSpec layout
gs = GridSpec(4, 4, figure=fig)

order = [['AFM'], [1], [0], [4, 'same', 6], [4, 'twin', 5], [2, 'inset', 3], ['hysteresis_loop']]

# List of axes indices in GridSpec for each subplot
subplot_specs = [(0, 2, 0, 2), # a (3D AFM)
                 (1, 2, 3, 4), # b (resonance frequency)
                 (1, 2, 2, 3), # c (waveform)
                 (0, 1, 2, 3), # d (real/imag)
                 (0, 1, 3, 4), # e (amp/phase)
                 (2, 4, 0, 2), # f (triangular waveform)
                 (2, 4, 2, 4)] # g (hysteresis)

# Create and set up subplots
for i, (r_start, r_end, c_start, c_end) in enumerate(subplot_specs):
    ax = fig.add_subplot(gs[r_start:r_end, c_start:c_end])
    if i < len(axes):
        idx = order[i]
        if idx[0] == 'AFM':
            image_path = "./Figures/AFM_tip_3D.jpeg"
            img = mpimg.imread(image_path)
            ax.imshow(img)
            ax.axis('off')  # Turn off axis for image subplot
        elif idx[0] == 'hysteresis_loop':
            raw_hysteresis_loop, voltage = dataset.get_hysteresis(
                                loop_interpolated=True, plotting_values=True)
            row = random.randint(0, 59)
            col = random.randint(0, 59)
            cycle = random.randint(0, 3)
            ax.plot(voltage.squeeze()*-1,
                       raw_hysteresis_loop[row, col, cycle, :].squeeze())
            ax.set_xlabel("Amplitude (Arb. U.)")
            ax.set_ylabel("Voltage (V)")
        elif len(idx) == 1:
            copy_axes_properties(axes[idx[0]], ax)
        else:
            copy_axes_properties(axes[idx[0]], ax, axes[idx[2]], idx[1])
            

# Adjust the spacing between the plots as needed
plt.tight_layout()

# Show the layout
plt.show()

**Figure 1**: Schematic diagram of band-excitation piezoresponse force microscopy switching spectroscopy (BE-SS) **a** Artistic render of an AFM tip applying an electric field to the surface. **b** Band
of excited frequencies excited. The dashed line shows the cantilever resonance frequency. **c** Bandexcitation waveform used to excite the cantilever in time domain **d** Fast Fourier transform of a
single-cantilever resonance during band-excitation piezoresponse force microscopy – shown as real
and imaginary components. **e** Magnitude spectrum showing the amplitude and the phase of cantilever
resonance. **f** Bipolar-triangular waveform used to switch the ferroelectric. The inset shows where the
band-excitation waveform was applied in both the voltage-on and voltage-off states. **g** Example of a
typical piezoelectric hysteresis loop obtained during BE-SS.

## Figure 3

In [ ]:
random_seed(seed=42)

postprocessor = ComplexPostProcessor(dataset)

model_ = Multiscale1DFitter(SHO_fit_func_nn, # function 
                            dataset.frequency_bin, # x data
                            2, # input channels
                            4, # output channels
                            dataset.SHO_scaler, 
                            postprocessor)

# instantiate the model
model = Model(model_, dataset, training=False, model_basename="SHO_Fitter_original_data")

model.load(
    "./Trained Models/SHO Fitter/SHO_Fitter_original_data_model_optimizer_Adam_epoch_4_train_loss_0.03404089391780737.pth"
)

# model.load_state_dict(torch.load("./Trained Models/SHO Fitter/SHO_Fitter_original_data_model_optimizer_Trust Region CG_epoch_599_train_loss_0.005903387442231178.pth"))

X_data, Y_data = dataset.NN_data()

# you can view the test and training dataset by replacing X_data with X_test or X_train
pred_data, scaled_param, parm = model.predict(X_data)

fig = BE_viz.SHO_switching_maps(parm, filename="Figure_15_NN_Switching_Maps")

In [ ]:
axes = fig.axes

In [ ]:
true_state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
    "measurement_state": "all",
}


fig = BE_viz.violin_plot_comparison(true_state, model, X_data, filename="Figure_16_Violin") 

In [ ]:
axes.extend(fig.axes)

In [ ]:
# sets the phase shift of the dataset
dataset.NN_phase_shift = np.pi/2
dataset.LSQF_phase_shift = np.pi/2
dataset.measurement_state = "all"

# sets the true state which to compare the results.
true_state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
    "measurement_state": "all",
}

# sets the state of the output data
out_state = {"scaled": True, "raw_format": "magnitude spectrum"}

# sets the number of examples to get
n = 1

LSQF = BE_viz.get_best_median_worst(
    true_state,
    prediction={"fitter": "LSQF"},
    out_state=out_state,
    SHO_results=True,
    n=n,
)

NN = BE_viz.get_best_median_worst(
    true_state, prediction=model, out_state=out_state, SHO_results=True, n=n
)

data = (LSQF, NN)
names = ["LSQF", "NN"]

fig = BE_viz.SHO_Fit_comparison(
    data,
    names,
    model_comparison=[model, {"fitter": "LSQF"}],
    out_state=out_state,
    filename="Figure_14_LSQF_NN_bmw_comparison",
    # display_results = None
)

In [ ]:
axes.extend(fig.axes)

In [ ]:
import pandas as pd
import seaborn as sns
import itertools

def copy_axes_properties(source_ax, target_ax, secondary_ax=None, type_ax=None):
    """Copy properties and data from source_ax to target_ax."""
    # Copy basic properties
    target_ax.set_xlabel(source_ax.get_xlabel())
    target_ax.set_ylabel(source_ax.get_ylabel())
    target_ax.set_xlim(source_ax.get_xlim())
    target_ax.set_ylim(source_ax.get_ylim())
    target_ax.set_title(source_ax.get_title())

    # Copy lines from the primary axis
    for line in source_ax.get_lines():
        label = line.get_label() if line.get_label() != '_nolegend_' else None
        # Copying line properties like color, linestyle, marker, etc.
        target_ax.plot(line.get_xdata(), line.get_ydata(), color=line.get_color(),
                       linestyle=line.get_linestyle(), marker=line.get_marker(), label=label)

    # Handle twin axes if present
    if secondary_ax:
        ax_twin = target_ax.twinx()
        ax_twin.set_ylim(secondary_ax.get_ylim())
        ax_twin.set_ylabel(secondary_ax.get_ylabel())

        for line in secondary_ax.get_lines():
            label = line.get_label() if line.get_label() != '_nolegend_' else None
            # Copying line properties for the twin axis
            ax_twin.plot(line.get_xdata(), line.get_ydata(), color=line.get_color(),
                         linestyle=line.get_linestyle(), marker=line.get_marker(), label=label)

    # Copy legends
    if source_ax.get_legend():
        target_ax.legend(fontsize='large')

    if secondary_ax and secondary_ax.get_legend():
        ax_twin.legend(fontsize='large')

# Create a figure
fig = plt.figure(figsize=(24, 24))

# Define the GridSpec layout
gs = GridSpec(6, 4, figure=fig)

order = [[42, 'twin', 48],
         [43, 'twin', 49],
         [44, 'twin', 50],
         [45, 'twin', 51],
         [46, 'twin', 52],
         [47, 'twin', 53],
         ['violin'],
        ]

# List of axes indices in GridSpec for each subplot
subplot_specs = [(0, 1, 0, 1), # a
                 (0, 1, 1, 2), # b
                 (1, 2, 0, 1), # c
                 (1, 2, 1, 2), # d
                 (2, 3, 0, 1), # e
                 (2, 3, 1, 2), # f
                 (0, 3, 2, 6), # g
                ]

# Create and set up subplots
for i, (r_start, r_end, c_start, c_end) in enumerate(subplot_specs):
    ax = fig.add_subplot(gs[r_start:r_end, c_start:c_end])
    if i < len(axes):
        idx = order[i]
        if idx[0] == 'violin':
            df = pd.DataFrame()

            # scales the parameters
            scaled_param = dataset.SHO_scaler.transform(parm)

            # gets the parameters from the SHO LSQF fit
            true = dataset.SHO_fit_results().reshape(-1, 4)

            # Builds the dataframe for the violin plot
            true_df = pd.DataFrame(
                true, columns=["Amplitude", "Resonance", "Q-Factor", "Phase"]
            )
            predicted_df = pd.DataFrame(
                scaled_param, columns=["Amplitude",
                                       "Resonance", "Q-Factor", "Phase"]
            )

            # merges the two dataframes
            df = pd.concat((true_df, predicted_df))

            # adds the labels to the dataframe
            names = [true, scaled_param]
            names_str = ["LSQF", "NN"]
            labels = ["A", "\u03C9", "Q", "\u03C6"]

            # adds the labels to the dataframe
            for j, name in enumerate(names):
                for i, label in enumerate(labels):
                    dict_ = {
                        "value": name[:, i],
                        "parameter": np.repeat(label, name.shape[0]),
                        "dataset": np.repeat(names_str[j], name.shape[0]),
                    }

                    df = pd.concat((df, pd.DataFrame(dict_)))

            # plots the data
            sns.violinplot(
                data=df, x="parameter", y="value", hue="dataset", split=True, ax=ax
            )

            # labels the figure and does some styling
            labelfigs(ax, 0, style="b")
            ax.set_ylabel("Scaled SHO Results")
            ax.set_xlabel("")

            # Get the legend associated with the plot
            legend = ax.get_legend()
            legend.set_title("")
            plt.setp(legend.get_texts(), fontsize='large') # Set the label size
        else:
            copy_axes_properties(axes[idx[0]], ax, axes[idx[2]], idx[1])
            

# Adjust the spacing between the plots as needed
plt.tight_layout()

# Show the layout
plt.show()

fig = BE_viz.SHO_switching_maps(parm);

# Set the figure size with 24 inches width
fig.set_size_inches(24, 10, forward=True)

# Show the layout
fig.show()

Figure 3. SHO fitting results of DNN in comparison with LSQF method’s results. a,c,e Best, median and worst predictions of LSQF method. b,d,f Best, median and worst predictions of neural network trained with ADAHESSIAN. g Distributions of predicted parameters. h Band-excitation waveform with switching voltage comparing maps of four parameters (i-q) predicted by NN and LSQF

## Figure 4

In [ ]:
h5_loop_fit, h5_loop_group = dataset.LSQF_Loop_Fit()

In [ ]:
# instantiate the visualization object
image_scalebar = [2000, 500, "nm", "br"]

# instantiate the dataset object
dataset = BE_Dataset(data_path)

BE_viz = Viz(dataset, printing, verbose=True, 
             SHO_ranges = [(0,1.5e-4), (1.31e6, 1.33e6), (-300, 300), (-np.pi, np.pi)], 
             image_scalebar = image_scalebar)

In [ ]:
from m3_learning.be.loop_fitter import loop_fitting_function_torch
from m3_learning.optimizers.TrustRegion import TRCG
import torch.optim as optim


data, voltage = dataset.get_hysteresis(scaled=True, loop_interpolated = True)
# V = np.swapaxes(np.atleast_2d(dataset.get_voltage), 0, 1).astype(np.float64)


model_ = Multiscale1DFitter(loop_fitting_function_torch, # function 
                            voltage[:,0].squeeze(), # x data
#                             V.squeeze(),
                            1, # input channels
                            9, # output parameters
                            dataset.loop_param_scaler,
                            loops_scaler=dataset.hysteresis_scaler)

# instantiate the model
model = Model(model_, dataset, training=True, model_basename="SHO_Fitter_original_data")

from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(data.reshape(-1,96), test_size=0.2, random_state=42, shuffle=True)

X_train = np.atleast_3d(X_train)

optimizer = {
    "name": "TRCG", 
    "optimizer": TRCG,
    "closure_size": 1,
    "cgopttol": 1e-3,
    "c0tr": 0.2,
    "c1tr": 0.25,
    "c2tr": 0.75,
    "t1tr": 0.75,
    "t2tr": 2.0,
    "radius_max": 5.0,  
    "radius_initial": 1.0,
    "radius" : 1.0,
    "device": "cuda",
    "ADAM_epochs": 100}


train = True

if train:
    # fits the model
    model.fit(
        X_train,
        1024,
        optimizer=optimizer,
        epochs = 500,
    )
else:
    model.load(
        "./Trained Models/SHO Fitter/SHO_Fitter_original_data_model_epoch_5_train_loss_0.0449272525189978.pth"
    )

In [ ]:
n = 1

data = ("LSQF", "NN")

fig = BE_viz.hysteresis_comparison(
    data,
    model=model,
    filename="Figure_XX_LSQF_NN_bmw_comparison",
)

In [ ]:
axes = fig.axes

In [ ]:
data, voltage = dataset.get_hysteresis(scaled=True, loop_interpolated = True)

fig = BE_viz.violin_plot_comparison_hysteresis(model,
                                         torch.atleast_3d(torch.tensor(data.reshape(-1, 96))),
                                         filename="Figure_XX_Violin") 

In [ ]:
axes.extend(fig.axes)

In [ ]:
data, voltage = dataset.get_hysteresis(scaled=True, loop_interpolated = True)
data = torch.atleast_3d(torch.tensor(data.reshape(-1, 96)))

pred_recon, pred_params_scaled, pred_params = model.predict(
    data,
    1024,
    translate_params=False,
    is_SHO=False
)

fig = BE_viz.hysteresis_maps(pred_params, cycle=0, filename="Figure_XX_NN_Hysteresis_Maps")

In [ ]:
import pandas as pd
import seaborn as sns
import itertools

def copy_axes_properties(source_ax, target_ax, secondary_ax=None, type_ax=None):
    """Copy properties and data from source_ax to target_ax."""
    # Copy basic properties
    target_ax.set_xlabel(source_ax.get_xlabel())
    target_ax.set_ylabel(source_ax.get_ylabel())
    target_ax.set_xlim(source_ax.get_xlim())
    target_ax.set_ylim(source_ax.get_ylim())
    target_ax.set_title(source_ax.get_title())

    # Copy lines from the primary axis
    for line in source_ax.get_lines():
        label = line.get_label() if line.get_label() != '_nolegend_' else None
        # Copying line properties like color, linestyle, marker, etc.
        target_ax.plot(line.get_xdata(), line.get_ydata(), color=line.get_color(),
                       linestyle=line.get_linestyle(), marker=line.get_marker(), label=label)

    # Handle twin axes if present
    if secondary_ax:
        ax_twin = target_ax.twinx()
        ax_twin.set_ylim(secondary_ax.get_ylim())
        ax_twin.set_ylabel(secondary_ax.get_ylabel())

        for line in secondary_ax.get_lines():
            label = line.get_label() if line.get_label() != '_nolegend_' else None
            # Copying line properties for the twin axis
            ax_twin.plot(line.get_xdata(), line.get_ydata(), color=line.get_color(),
                         linestyle=line.get_linestyle(), marker=line.get_marker(), label=label)

    # Copy legends
    if source_ax.get_legend():
        target_ax.legend(fontsize='large')

    if secondary_ax and secondary_ax.get_legend():
        ax_twin.legend(fontsize='large')

# Create a figure
fig = plt.figure(figsize=(24, 24))

# Define the GridSpec layout
gs = GridSpec(6, 4, figure=fig)

order = [[0, 'twin', 6],
         [1, 'twin', 7],
         [2, 'twin', 8],
         [3, 'twin', 9],
         [4, 'twin', 10],
         [5, 'twin', 11],
         ['violin'],
        ]

# List of axes indices in GridSpec for each subplot
subplot_specs = [(0, 1, 0, 1), # a
                 (0, 1, 1, 2), # b
                 (1, 2, 0, 1), # c
                 (1, 2, 1, 2), # d
                 (2, 3, 0, 1), # e
                 (2, 3, 1, 2), # f
                 (0, 3, 2, 6), # g
                ]

# Create and set up subplots
for i, (r_start, r_end, c_start, c_end) in enumerate(subplot_specs):
    ax = fig.add_subplot(gs[r_start:r_end, c_start:c_end])
    if i < len(axes):
        idx = order[i]
        if idx[0] == 'violin':
            df = pd.DataFrame()

            # uses the model to get the predictions
            pred_data, scaled_param, params = model.predict(data, is_SHO=False)

            true = dataset.LSQF_hysteresis_params().reshape(-1, 9)

            true_scaled = dataset.loop_param_scaler.transform(true)

            # Builds the dataframe for the violin plot
            true_df = pd.DataFrame(
                true, columns=["a0", "a1", "a2", "a3", "a4",
                               "b0", "b1", "b2", "b3"]
            )
            predicted_df = pd.DataFrame(
                scaled_param, columns=["a0", "a1", "a2", "a3", "a4",
                                       "b0", "b1", "b2", "b3"]
            )

            # merges the two dataframes
            df = pd.concat((predicted_df, true_df))

            # adds the labels to the dataframe
            names = [true_scaled, scaled_param]
            names_str = ["NN", "LSQF"]
            labels = ["a0", "a1", "a2", "a3", "a4", "b0", "b1", "b2", "b3"]

            # adds the labels to the dataframe
            for j, name in enumerate(names):
                for i, label in enumerate(labels):
                    dict_ = {
                        "value": name[:, i],
                        "parameter": np.repeat(label, name.shape[0]),
                        "dataset": np.repeat(names_str[j], name.shape[0]),
                    }

                    df = pd.concat((df, pd.DataFrame(dict_)))

#             # builds the plot
#             fig, ax = plt.subplots(figsize=(4, 4))

            # plots the data
            sns.violinplot(
                data=df, x="parameter", y="value", hue="dataset", split=True, ax=ax
            )

            # labels the figure and does some styling
            labelfigs(ax, 0, style="b")
            ax.set_ylabel("Scaled Hysteresis Results")
            ax.set_xlabel("")

            # Get the legend associated with the plot
            legend = ax.get_legend()
            legend.set_title("")
            plt.setp(legend.get_texts(), fontsize='large') # Set the label size
        else:
            copy_axes_properties(axes[idx[0]], ax, axes[idx[2]], idx[1])
            

# Adjust the spacing between the plots as needed
plt.tight_layout()

# Show the layout
plt.show()

data, voltage = dataset.get_hysteresis(scaled=True, loop_interpolated = True)
data = torch.atleast_3d(torch.tensor(data.reshape(-1, 96)))

pred_recon, pred_params_scaled, pred_params = model.predict(
    data,
    1024,
    translate_params=False,
    is_SHO=False
)

fig = BE_viz.hysteresis_maps(pred_params, cycle=0);

# Set the figure size with 24 inches width
fig.set_size_inches(24, 10, forward=True)

# Show the layout
fig.show()

Figure 4. Piezoelectric hysteresis loops fitting results of DNN in comparison with LSQF method’s results. a,c,e Best, median, and worst predictions of LSQF method. b,d,f Best, median, and worst predictions of neural network trained with Trust Region CG. g Distributions of predicted parameters. h Color maps of the signal of parameters resulted from LSQF method. i Color maps of the signal of parameters resulted from neural network.

## Figure 5